Note, the data has to be in the current folder, otherwise it won't work.

In [11]:
import numpy as np
from sklearn.metrics import jaccard_score
from torch import from_numpy
from tqdm import tqdm

from segmentation.utils import SegmentationDataset, train
import torch
from segmentation.resnet_18_fcn import ResNet18FCN
from unet import UNet

In [74]:
unet = UNet(in_channels=3, out_channels=13)

trainset = SegmentationDataset(train=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=4)

testset = SegmentationDataset(train=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=4)
train(net=unet, train_data=[trainset, trainloader])

1/2: 100%|██████████| 4000/4000 [05:28<00:00, 12.16img/s, loss=0.182]


In [4]:
net = ResNet18FCN(in_channels=3, out_channels=13)
trainset = SegmentationDataset(train=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=4)

testset = SegmentationDataset(train=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=4)
train(net=net, train_data=[trainset, trainloader])

Value of meanIOU metric on the test set for the FCN-32

In [76]:
actual, predicted = [], []
for i in tqdm(range(len(testset))):
    batch = testset[i]
    img = batch['image']
    labels = batch['mask']
    img_tensor = from_numpy(img).cuda()
    img_tensor = img_tensor.unsqueeze(0)
    outputs = net(img_tensor)
    out_image = outputs[0, :, :, :].detach().cpu().numpy()
    seg_mask = np.argmax(out_image, axis=0)

    actual.append(np.asarray(labels[0, :, :]).flatten())
    predicted.append(seg_mask.flatten())

np.mean([jaccard_score(predicted[i], actual[i], average='macro') for i in tqdm(range(len(predicted)))])

100%|██████████| 1000/1000 [00:32<00:00, 30.66it/s]


0.4150983204258001

Value of meanIOU metric on the test set for the U-Net

In [75]:
actual, predicted = [], []
for i in tqdm(range(len(testset))):
    batch = testset[i]
    img = batch['image']
    labels = batch['mask']
    img_tensor = from_numpy(img).cuda()
    img_tensor = img_tensor.unsqueeze(0)
    outputs = unet(img_tensor)
    out_image = outputs[0, :, :, :].detach().cpu().numpy()
    seg_mask = np.argmax(out_image, axis=0)

    actual.append(np.asarray(labels[0, :, :]).flatten())
    predicted.append(seg_mask.flatten())

np.mean([jaccard_score(predicted[i], actual[i], average='macro') for i in tqdm(range(len(predicted)))])

100%|██████████| 1000/1000 [00:19<00:00, 52.15it/s]


0.5120416028745854